In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,FunctionTransformer
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.linear_model import Ridge,ElasticNet,Lasso,LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [2]:
df = pd.read_csv('/Users/suryasaikadali/Downloads/pw_skills/end_to_End_projects/student_performance/artifacts/data.csv')

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race_ethnicity               1000 non-null   object
 2   parental_level_of_education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test_preparation_course      1000 non-null   object
 5   math_score                   1000 non-null   int64 
 6   reading_score                1000 non-null   int64 
 7   writing_score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [5]:
x = df.drop('math_score',axis = 1)
y = df['math_score']

In [6]:
cat_cols = [i for i in x.columns if df[i].dtype=='object']
num_cols = [i for i in x.columns if df[i].dtype=='int64']

In [7]:
cat_cols

['gender',
 'race_ethnicity',
 'parental_level_of_education',
 'lunch',
 'test_preparation_course']

In [8]:
num_cols

['reading_score', 'writing_score']

In [9]:
num_pipe = Pipeline(
    steps = [
        ('scaler',StandardScaler())
    ]
)

cat_pipe = Pipeline(
    steps = [
        ('scaler',StandardScaler())
    ]
)

In [10]:
preprocessor = ColumnTransformer([
    ('num',num_pipe,num_cols),
    ('cat',cat_pipe,cat_cols)
  
])
 

In [11]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [12]:
le = LabelEncoder()

In [13]:
for i in cat_cols:
    x_train[i] = le.fit_transform(x_train[i])
    x_test[i] = le.transform(x_test[i])

In [14]:
x_train

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
82,1,0,4,0,0,47,54
991,0,1,5,1,0,82,78
789,0,2,3,0,1,65,61
894,0,4,0,1,1,62,69
398,1,1,5,1,1,63,57
...,...,...,...,...,...,...,...
106,0,3,3,1,1,100,100
270,1,2,1,1,1,63,61
860,0,2,0,1,1,62,53
435,1,2,4,0,0,48,53


In [15]:
x_test

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
521,0,2,0,1,1,86,84
737,0,1,4,0,0,66,73
740,1,3,1,1,1,73,72
660,1,2,4,0,1,77,73
411,1,4,4,1,0,83,78
...,...,...,...,...,...,...,...
109,0,1,5,1,1,64,72
430,1,2,0,0,1,66,59
77,1,0,1,1,0,78,81
84,1,3,2,0,1,39,34


In [16]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train), columns = preprocessor.get_feature_names_out())
x_test = pd.DataFrame(preprocessor.transform(x_test), columns = preprocessor.get_feature_names_out())

In [17]:
x_train

,num__reading_score,num__writing_score,cat__gender,cat__race_ethnicity,cat__parental_level_of_education,cat__lunch,cat__test_preparation_course
0,-1.594536,-0.988448,1.063335,-1.891188,0.847306,-1.368782,-1.356801
1,0.857519,0.628865,-0.940437,-1.027369,1.396554,0.730577,-1.356801
2,-0.333479,-0.516732,-0.940437,-0.163550,0.298058,-1.368782,0.737028
3,-0.543656,0.022373,-0.940437,1.564089,-1.349685,0.730577,0.737028
4,-0.473597,-0.786284,1.063335,-1.027369,1.396554,0.730577,0.737028
...,...,...,...,...,...,...,...
745,2.118575,2.111403,-0.940437,0.700269,0.298058,0.730577,0.737028
746,-0.473597,-0.516732,1.063335,-0.163550,-0.800437,0.730577,0.737028
747,-0.543656,-1.055836,-0.940437,-0.163550,-1.349685,0.730577,0.737028
748,-1.524478,-1.055836,1.063335,-0.163550,0.847306,-1.368782,-1.356801


In [18]:
models = {
    'linear regressor':LinearRegression(),
    'random forest Regressor':RandomForestRegressor(),
    'xgboost Regressor':XGBRegressor(),
    'ridge':Ridge(),
    'lasso':Lasso(),
    'elastic net':ElasticNet()
}

In [19]:
for i,j in models.items():
    print(i)
    print('&&&&&&')
    print(j)

linear regressor
&&&&&&
LinearRegression()
random forest Regressor
&&&&&&
RandomForestRegressor()
xgboost Regressor
&&&&&&
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)
ridge
&&&&&&
Ridge()
lasso
&&&&&&
Lasso()
elastic net
&&&&&&
ElasticNet()


In [20]:
def evaluate_model(true,predicted):
    mse = mean_squared_error(true,predicted)
    mae = mean_absolute_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2 = r2_score(true,predicted)
    return mae,rmse,r2

In [21]:
r2_list = []

In [22]:
for key,values in models.items():
    model = values.fit(x_train,y_train)
        
    y_pred = model.predict(x_test)

    mae,rmse,r2 = evaluate_model(y_test,y_pred)
    r2_list.append(f'{key}:{r2}')    
    print(f'The rsquare score of {key} model is {r2}')
    print(f'The mae  of {key} model is {mae}')
    print(f'The rmse of {key} model is {rmse}')
    print('%%%%%%%%%%%%%%%')
    print('\n')
        

The rsquare score of linear regressor model is 0.8795296105895843
The mae  of linear regressor model is 4.315631383102993
The rmse of linear regressor model is 5.4441317454034595
%%%%%%%%%%%%%%%


The rsquare score of random forest Regressor model is 0.8495502871040593
The mae  of random forest Regressor model is 4.7521
The rmse of random forest Regressor model is 6.083929193276183
%%%%%%%%%%%%%%%


The rsquare score of xgboost Regressor model is 0.8270697593688965
The mae  of xgboost Regressor model is 5.104663970947265
The rmse of xgboost Regressor model is 6.522647607132915
%%%%%%%%%%%%%%%


The rsquare score of ridge model is 0.8795552728979009
The mae  of ridge model is 4.314547588170144
The rmse of ridge model is 5.443551866692351
%%%%%%%%%%%%%%%


The rsquare score of lasso model is 0.8490051926006673
The mae  of lasso model is 4.809818828061381
The rmse of lasso model is 6.094940573392981
%%%%%%%%%%%%%%%


The rsquare score of elastic net model is 0.787650738358463
The mae  of 